# Elixir: typespecs & behaviors

### Types & specs

Elixir is a dynamically typed language, so all types in Elixir are inferred by the runtime. Nonetheless, Elixir comes with typespecs, which are a notation used for:

- declaring typed function signatures (specifications);
- declaring custom data types.

### Function specs

Elixir provides some basic types, such as integer or pid, as well as more complex types: for example, the round/1 function, which rounds a float to its nearest integer, takes a number as an argument (an integer or a float) and returns an integer. As you can see in [its documentation](https://hexdocs.pm/elixir/Kernel.html#round/1), round/1’s typed signature is written as:

    ```round(number) :: integer```
    
```::``` means that the function on the left side returns a value whose type is what’s on the right side. Function specs are written with the ```@spec``` directive, placed right before the function definition. ```round/1``` could be written as:

    ```@spec round(number) :: integer
    def round(number), do: # implementation...```
    
Elixir also support compound types. For example a list of integers has type ```[integer]```. Elixir's built-in types are documented [here](https://hexdocs.pm/elixir/typespecs.html)

### Custom types
While Elixir provides a lot of useful built-in types, it’s convenient to define custom types when appropriate. This can be done when defining modules through the ```@type``` directive.

Say we have a ```LousyCalculator``` module, which, instead of returning numbers, it returns tuples with the result of an operation as the first element and a random remark as the second element.

In [1]:
defmodule LousyCalculator do
  @spec add(number, number) :: {number, String.t}
  def add(x, y), do: {x + y, "You need a calculator to do that?!"}

  @spec multiply(number, number) :: {number, String.t}
  def multiply(x, y), do: {x * y, "Jeez, come on!"}
end

{:module, LousyCalculator, <<70, 79, 82, 49, 0, 0, 5, 152, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 146, 0, 0, 0, 16, 22, 69, 108, 105, 120, 105, 114, 46, 76, 111, 117, 115, 121, 67, 97, 108, 99, 117, 108, 97, 116, 111, 114, ...>>, {:multiply, 2}}

Tuples are a compound type and each tuple is identified by the types inside it. To understand why String.t is not written as string, have another look at the [typespecs docs](https://hexdocs.pm/elixir/typespecs.html#the-string-type).

Defining function specs this way quickly becomes annoying since we’re repeating the type ```{number, String.t}``` over and over. We can use the ```@type``` directive in order to declare our own custom type.

In [2]:
defmodule LousyCalculator do
  @typedoc """
  Just a number followed by a string.
  """
  @type number_with_remark :: {number, String.t}

  @spec add(number, number) :: number_with_remark
  def add(x, y), do: {x + y, "You need a calculator to do that?"}

  @spec multiply(number, number) :: number_with_remark
  def multiply(x, y), do: {x * y, "It is like addition on steroids."}
end

  nofile:1



{:module, LousyCalculator, <<70, 79, 82, 49, 0, 0, 6, 16, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 146, 0, 0, 0, 16, 22, 69, 108, 105, 120, 105, 114, 46, 76, 111, 117, 115, 121, 67, 97, 108, 99, 117, 108, 97, 116, 111, 114, ...>>, {:multiply, 2}}

The ```@typedoc``` directive, similar to ```@doc``` and ```@moduledoc```, is used to document custom types.

Custom types defined through ```@type``` are exported and available outside the module they’re defined in.

In [3]:
defmodule QuietCalculator do
  @spec add(number, number) :: number
  def add(x, y), do: make_quiet(LousyCalculator.add(x, y))

  @spec make_quiet(LousyCalculator.number_with_remark) :: number
  defp make_quiet({num, _remark}), do: num
end

{:module, QuietCalculator, <<70, 79, 82, 49, 0, 0, 5, 0, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 167, 0, 0, 0, 15, 22, 69, 108, 105, 120, 105, 114, 46, 81, 117, 105, 101, 116, 67, 97, 108, 99, 117, 108, 97, 116, 111, 114, ...>>, {:make_quiet, 1}}

If you want to keep a custom type private, use ```@typep``` instead of ```@type```.

### Static code analysis

Typespecs are not only useful to developers as additional documentation. The Erlang tool [Dialyzer](http://www.erlang.org/doc/man/dialyzer.html), for example, uses typespecs in order to perform static analysis of code. That’s why, in the ```QuietCalculator``` example, we wrote a spec for the make_quiet/1 function even though it was defined as a private function.

### Behaviors

Many modules share the same public API. Take a look at [Plug](https://github.com/elixir-lang/plug), which is a specification for composable modules in web applications. Each plug is a module which has to implement at least two public functions: ```init/1``` and ```call/2```.

Behaviours provide a way to:

- define a set of functions that have to be implemented by a module;
- ensure that a module implements all the functions in that set.

If you have to, you can think of behaviours like interfaces in object oriented languages like Java: a set of function signatures that a module has to implement.

### Defining behaviors

Say we want to implement a bunch of parsers, each parsing structured data: for example, a JSON parser and a MessagePack parser. Each of these two parsers will behave the same way: both provide a ```parse/1``` function and an ```extensions/0``` function. 

```parse/1``` will return an Elixir representation of the structured data; ```extensions/0``` will return a list of file extensions that can be used for each type of data (e.g., .json for JSON files).

We can create a ```Parser``` behaviour:

In [4]:
defmodule Parser do
  @callback parse(String.t) :: {:ok, term} | {:error, String.t}
  @callback extensions() :: [String.t]
end

{:module, Parser, <<70, 79, 82, 49, 0, 0, 4, 144, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 164, 0, 0, 0, 15, 13, 69, 108, 105, 120, 105, 114, 46, 80, 97, 114, 115, 101, 114, 8, 95, 95, 105, 110, 102, 111, 95, 95, ...>>, :ok}

Modules adopting the ```Parser``` behaviour will have to implement all the functions defined with the ```@callback``` directive. As you can see, ```@callback``` expects a function name but also a function specification like the ones used with the ```@spec``` directive we saw above. Also note that the ```term``` type is used to represent the parsed value. In Elixir, the `term` type is a shortcut to represent any type.

### Adopting behaviors

In [5]:
defmodule JSONParser do
  @behaviour Parser

  @impl Parser
  def parse(str), do: {:ok, "some json " <> str} # ... parse JSON
  
  @impl Parser
  def extensions, do: ["json"]
end

{:module, JSONParser, <<70, 79, 82, 49, 0, 0, 5, 48, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 158, 0, 0, 0, 17, 17, 69, 108, 105, 120, 105, 114, 46, 74, 83, 79, 78, 80, 97, 114, 115, 101, 114, 8, 95, 95, 105, 110, ...>>, {:extensions, 0}}

In [6]:
defmodule YAMLParser do
  @behaviour Parser

  @impl Parser
  def parse(str), do: {:ok, "some yaml " <> str} # ... parse YAML
  
  @impl Parser
  def extensions, do: ["yml"]
end

{:module, YAMLParser, <<70, 79, 82, 49, 0, 0, 5, 48, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 158, 0, 0, 0, 17, 17, 69, 108, 105, 120, 105, 114, 46, 89, 65, 77, 76, 80, 97, 114, 115, 101, 114, 8, 95, 95, 105, 110, ...>>, {:extensions, 0}}

If a module adopting a given behaviour doesn’t implement one of the callbacks required by that behaviour, a compile-time warning will be generated.

With `@impl` you can ensure that you are implementing the correct callbacks from the given behaviour in an explicit manner. For example, the following parser implements both `parse` and `extensions`, however thanks to a typo, `BADParser` implements `parse/0` instead of `parse/1`.

In [7]:
defmodule BADParser do
  @behaviour Parser

  @impl Parser
  def parse, do: {:ok, "something bad"}
  
  @impl Parser
  def extensions, do: ["bad"]
end


  * Parser.extensions/0 (function)
  * Parser.parse/1 (function)

  nofile:5

  nofile:1



{:module, BADParser, <<70, 79, 82, 49, 0, 0, 4, 180, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 140, 0, 0, 0, 14, 16, 69, 108, 105, 120, 105, 114, 46, 66, 65, 68, 80, 97, 114, 115, 101, 114, 8, 95, 95, 105, 110, 102, ...>>, {:extensions, 0}}

This code generates a warning: you are mistakenly implementing `parse/0` instead of `parse/1`. You can read more about `@impl` in the module documentation.

### Dynamic dispatch

Behaviours are frequently used with dynamic dispatching. For example, we could add a `parse!` function to the `Parser` module that dispatches to the given implementation and returns `:ok` or raises in cases of `:error`:

In [8]:
defmodule Parser do
  @callback parse(String.t) :: {:ok, term} | {:error, String.t}
  @callback extensions() :: [String.t]

  def parse!(implementation, contents) do
    case implementation.parse(contents) do
      {:ok, data} -> data
      {:error, error} -> raise ArgumentError, "parsing error: #{error}"
    end
  end
end

  nofile:1



{:module, Parser, <<70, 79, 82, 49, 0, 0, 7, 36, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 5, 0, 0, 0, 25, 13, 69, 108, 105, 120, 105, 114, 46, 80, 97, 114, 115, 101, 114, 8, 95, 95, 105, 110, 102, 111, 95, 95, ...>>, {:parse!, 2}}